In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
csvfile = pd.read_csv("Database3.csv")
csvfile_a = pd.read_csv("Database3a - 5.csv")
features = csvfile.columns
d = len(features) 
X = csvfile.iloc[:, ].values
answer = csvfile_a.iloc[:, ].values

snum = 0.0000001
initial_preference_vector = np.repeat(snum, d)


In [2]:
def choice(items, top_k, k):
    for j in items:
        for i in range(0, k):
            if (np.array_equal(answer[i], top_k[j])):
                return j
    return False  

In [3]:
def user_items(comb, all_vecs, terminate):
    vec = []
    if (all_vecs == []):
        vec = comb[0]
        all_vecs.append(comb[0])
        terminate = False
        return vec, all_vecs, terminate 
    else: 
        for i in comb:
            k = 0
            for j in all_vecs:
                if (np.array_equal(i, j)):
                    continue
                else:
                    k += 1
            if (k == len(all_vecs)):
                vec = i
                all_vecs.append(list(vec))
                terminate = False
                return list(vec), all_vecs, terminate  
                    
    return list(vec), (all_vecs), terminate  

In [4]:
def combination(vec, n):
    if not n:
        return [[]]
    if not vec:
        return []
    head = [vec[0]]
    tail = vec[1:]
    new_comb = list([head + lis for lis in combination(tail, n - 1)])
    return new_comb + combination(tail, n)

In [5]:
def preference_learning(k, c, preference_vector, X, all_vecs): 
    distances = np.linalg.norm(X - preference_vector, axis = 1)
    top_k = distances.argsort()[:k]

    x = StandardScaler().fit_transform(X[top_k])
    pca_item = PCA(n_components = k - 1)
    principalComponents_item = pca_item.fit_transform(x)
     
    vec = list(np.argmax(principalComponents_item, axis = 0))

    comb = list(combination(vec, c))
    it = 0
    for i in comb:    
        if (np.array_equal(pd.unique(i), i)):
            it += 1
        else:
            comb = np.delete(comb, it, 0)
    new_comb = []
    for i in comb:
        vec_comb = []
        for j in i:
            vec_comb.append(j)
        new_comb.append(vec_comb)
    
    terminate = True
    vec2, new_all_vecs, new_terminate = user_items(comb, all_vecs, terminate)
    
    vec_index = choice(vec2, X[top_k], k)
    v = X[top_k][vec_index]
    new_preference_vector = (preference_vector + v) / 2

    new_distances = np.linalg.norm(X - new_preference_vector, axis = 1)
    new_top_k = new_distances.argsort()[:k]
    return X[new_top_k], (new_preference_vector), v, (new_all_vecs), new_terminate, (new_comb), vec

In [6]:
def compare(a, b): 
    m = 0
    for i in a:
        for j in b:
            if (np.array_equal(i, j)):
                m += 1
    if (m == len(a)):
        return True
    else: return False

In [7]:
# Testing Framework
iterations = 0
all_vecs = []

while True:
    if (iterations > 1):
        break
        
    if (iterations == 0):
        a, pv, v, av, t, comb, vec = preference_learning(5, 3, initial_preference_vector, X, all_vecs)
    else:
        a, pv, v, av, t, comb, vec = preference_learning(5, 3, pv, X, av)
        
    if (compare(a, answer)):
        iterations += 1
        break # Not in Order
    else:
        # print(v)
        iterations += 1
print(a)
print(answer)
print(iterations)
print(av)
print(pv)
print(t)
print(comb)
print(vec)

[[1 1 1 1 2 2 1 1 1]
 [1 1 1 1 1 2 2 1 1]
 [1 1 1 2 2 1 1 1 1]
 [2 1 1 1 1 1 2 1 1]
 [1 2 2 1 1 1 1 1 1]]
[[1 1 1 1 2 2 1 1 1]
 [1 1 1 1 1 2 2 1 1]
 [2 1 1 1 1 1 2 1 1]
 [1 1 1 1 1 1 3 3 1]
 [1 1 1 1 1 1 1 3 3]]
2
[array([4, 2, 3], dtype=int64), [1, 2, 0]]
[0.75000003 0.75000003 0.75000003 0.75000003 1.25000003 1.50000002
 1.00000002 0.75000003 0.75000003]
False
[[1, 2, 0], [1, 0, 2]]
[1, 2, 0, 2]
